<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Norbert", "transactions": [{"transaction-id": 378, "amount": 1116}, {"transaction-id": 581, "amount": 1211}, {"transaction-id": 818, "amount": 1149}, {"transaction-id": 1124, "amount": 1250}, {"transaction-id": 1375, "amount": 1371}, {"transaction-id": 2449, "amount": 1130}, {"transaction-id": 2513, "amount": 1269}, {"transaction-id": 2551, "amount": 1132}, {"transaction-id": 2593, "amount": 1167}, {"transaction-id": 3030, "amount": 1237}, {"transaction-id": 3292, "amount": 1334}, {"transaction-id": 3668, "amount": 1105}, {"transaction-id": 3735, "amount": 1140}, {"transaction-id": 4396, "amount": 1290}, {"transaction-id": 4673, "amount": 1439}, {"transaction-id": 6516, "amount": 1359}, {"transaction-id": 7708, "amount": 1227}, {"transaction-id": 7854, "amount": 1287}, {"transaction-id": 8299, "amount": 1195}, {"transaction-id": 8647, "amount": 1202}, {"transaction-id": 9250, "amount": 1212}, {"transaction-id": 9721, "amount": 1258}, {"transaction-id": 9792, "amoun

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Norbert", "transactions": [{"transaction-id": 378, "amount": 1116}, {"transaction-id": 581, "amount": 1211}, {"transaction-id": 818, "amount": 1149}, {"transaction-id": 1124, "amount": 1250}, {"transaction-id": 1375, "amount": 1371}, {"transaction-id": 2449, "amount": 1130}, {"transaction-id": 2513, "amount": 1269}, {"transaction-id": 2551, "amount": 1132}, {"transaction-id": 2593, "amount": 1167}, {"transaction-id": 3030, "amount": 1237}, {"transaction-id": 3292, "amount": 1334}, {"transaction-id": 3668, "amount": 1105}, {"transaction-id": 3735, "amount": 1140}, {"transaction-id": 4396, "amount": 1290}, {"transaction-id": 4673, "amount": 1439}, {"transaction-id": 6516, "amount": 1359}, {"transaction-id": 7708, "amount": 1227}, {"transaction-id": 7854, "amount": 1287}, {"transaction-id": 8299, "amount": 1195}, {"transaction-id": 8647, "amount": 1202}, {"transaction-id": 9250, "amount": 1212}, {"transaction-id": 9721, "amount": 1258}, {"transaction-id": 9792, "amoun

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Norbert',
  'transactions': [{'transaction-id': 378, 'amount': 1116},
   {'transaction-id': 581, 'amount': 1211},
   {'transaction-id': 818, 'amount': 1149},
   {'transaction-id': 1124, 'amount': 1250},
   {'transaction-id': 1375, 'amount': 1371},
   {'transaction-id': 2449, 'amount': 1130},
   {'transaction-id': 2513, 'amount': 1269},
   {'transaction-id': 2551, 'amount': 1132},
   {'transaction-id': 2593, 'amount': 1167},
   {'transaction-id': 3030, 'amount': 1237},
   {'transaction-id': 3292, 'amount': 1334},
   {'transaction-id': 3668, 'amount': 1105},
   {'transaction-id': 3735, 'amount': 1140},
   {'transaction-id': 4396, 'amount': 1290},
   {'transaction-id': 4673, 'amount': 1439},
   {'transaction-id': 6516, 'amount': 1359},
   {'transaction-id': 7708, 'amount': 1227},
   {'transaction-id': 7854, 'amount': 1287},
   {'transaction-id': 8299, 'amount': 1195},
   {'transaction-id': 8647, 'amount': 1202},
   {'transaction-id': 9250, 'amount': 1212},
   {'trans

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 2,
  'name': 'Alice',
  'transactions': [{'transaction-id': 590, 'amount': -1090},
   {'transaction-id': 598, 'amount': -910},
   {'transaction-id': 2207, 'amount': -374},
   {'transaction-id': 2445, 'amount': -664},
   {'transaction-id': 2592, 'amount': -980},
   {'transaction-id': 2718, 'amount': -852},
   {'transaction-id': 4763, 'amount': -603},
   {'transaction-id': 6118, 'amount': -411},
   {'transaction-id': 8593, 'amount': -1027},
   {'transaction-id': 8641, 'amount': -923},
   {'transaction-id': 9184, 'amount': -659},
   {'transaction-id': 9189, 'amount': -720}]},
 {'id': 5,
  'name': 'Alice',
  'transactions': [{'transaction-id': 482, 'amount': 13483},
   {'transaction-id': 605, 'amount': 14007},
   {'transaction-id': 688, 'amount': 13586},
   {'transaction-id': 705, 'amount': 13925},
   {'transaction-id': 748, 'amount': 13137},
   {'transaction-id': 1324, 'amount': 14043},
   {'transaction-id': 1752, 'amount': 13756},
   {'transaction-id': 2204, 'amount': 13731},
   

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 12},
 {'name': 'Alice', 'count': 44},
 {'name': 'Alice', 'count': 26},
 {'name': 'Alice', 'count': 33},
 {'name': 'Alice', 'count': 16})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(12, 44, 26, 33, 16)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

36.966

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 590, 'amount': -1090},
  {'transaction-id': 598, 'amount': -910},
  {'transaction-id': 2207, 'amount': -374},
  {'transaction-id': 2445, 'amount': -664},
  {'transaction-id': 2592, 'amount': -980},
  {'transaction-id': 2718, 'amount': -852},
  {'transaction-id': 4763, 'amount': -603},
  {'transaction-id': 6118, 'amount': -411},
  {'transaction-id': 8593, 'amount': -1027},
  {'transaction-id': 8641, 'amount': -923},
  {'transaction-id': 9184, 'amount': -659},
  {'transaction-id': 9189, 'amount': -720}],
 [{'transaction-id': 482, 'amount': 13483},
  {'transaction-id': 605, 'amount': 14007},
  {'transaction-id': 688, 'amount': 13586},
  {'transaction-id': 705, 'amount': 13925},
  {'transaction-id': 748, 'amount': 13137},
  {'transaction-id': 1324, 'amount': 14043},
  {'transaction-id': 1752, 'amount': 13756},
  {'transaction-id': 2204, 'amount': 13731},
  {'transaction-id': 2282, 'amount': 14030},
  {'transaction-id': 2337, 'amount': 13532},
  {'transaction-id': 2473,

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 590, 'amount': -1090},
 {'transaction-id': 598, 'amount': -910},
 {'transaction-id': 2207, 'amount': -374})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(-1090, -910, -374)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

2652.643672563978

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 110), ('Alice', 120), ('Alice', 130), ('Alice', 140), ('Bob', 110), ('Bob', 120), ('Bob', 130), ('Bob', 140), ('Charlie', 137), ('Charlie', 151), ('Charlie', 164), ('Charlie', 177), ('Dan', 124), ('Dan', 135), ('Dan', 223), ('Edith', 82), ('Edith', 92), ('Edith', 99), ('Edith', 101), ('Frank', 33), ('Frank', 36), ('Frank', 39), ('Frank', 42), ('George', 226), ('George', 245), ('George', 267), ('George', 289), ('Hannah', 129), ('Hannah', 142), ('Hannah', 155), ('Hannah', 167), ('Ingrid', 88), ('Ingrid', 96), ('Ingrid', 104), ('Ingrid', 112), ('Jerry', 124), ('Jerry', 132), ('Jerry', 143), ('Jerry', 153), ('Kevin', 110), ('Kevin', 120), ('Kevin', 130), ('Kevin', 140), ('Laura', 98), ('Laura', 104), ('Laura', 112), ('Laura', 125), ('Michael', 163), ('Michael', 178), ('Michael', 193), ('Michael', 207), ('Norbert', 88), ('Norbert', 93), ('Norbert', 100), ('Norbert', 109), ('Oliver', 120), ('Oliver', 132), ('Oliver', 143), ('Oliver', 153), ('Patricia', 76), ('Patricia', 84), ('Pat

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 500), ('Bob', 500), ('Charlie', 629), ('Dan', 482), ('Edith', 374), ('Frank', 150), ('George', 1027), ('Hannah', 593), ('Ingrid', 400), ('Jerry', 552), ('Kevin', 500), ('Laura', 439), ('Michael', 741), ('Norbert', 390), ('Oliver', 548), ('Patricia', 349), ('Quinn', 499), ('Ray', 300), ('Sarah', 448), ('Tim', 237), ('Ursula', 600), ('Victor', 607), ('Wendy', 350), ('Xavier', 383), ('Yvonne', 350), ('Zelda', 296)]
CPU times: user 119 ms, sys: 10.7 ms, total: 130 ms
Wall time: 558 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Norbert,"[{'transaction-id': 378, 'amount': 1116}, {'tr..."
1,1,Michael,"[{'transaction-id': 43, 'amount': 3369}, {'tra..."
2,2,Alice,"[{'transaction-id': 590, 'amount': -1090}, {'t..."
3,3,Charlie,"[{'transaction-id': 2367, 'amount': 717}, {'tr..."
4,4,George,"[{'transaction-id': 302, 'amount': 75}, {'tran..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 195 ms, sys: 16.6 ms, total: 211 ms
Wall time: 1.68 s


name
Alice      500
Bob        500
Charlie    629
Dan        482
Edith      374
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Norbert', 'amount': 1116, 'transaction-id': 378},
 {'id': 0, 'name': 'Norbert', 'amount': 1211, 'transaction-id': 581},
 {'id': 0, 'name': 'Norbert', 'amount': 1149, 'transaction-id': 818})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Norbert,1116,378
1,0,Norbert,1211,581
2,0,Norbert,1149,818
3,0,Norbert,1250,1124
4,0,Norbert,1371,1375


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 173 ms, sys: 13.2 ms, total: 186 ms
Wall time: 1.4 s


name
Alice       18483
Bob         23420
Charlie     25940
Dan         23589
Edith       15383
Frank        6075
George      47415
Hannah      42035
Ingrid      19238
Jerry       18444
Kevin       24317
Laura       11333
Michael     41161
Norbert     16727
Oliver       8228
Patricia    17012
Quinn       21973
Ray         11581
Sarah       16145
Tim          3878
Ursula      14798
Victor      17344
Wendy       22816
Xavier      11838
Yvonne       8545
Zelda       12282
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()